In [17]:
import dash
from dash import dcc, html, Input, Output, State, callback_context
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# ==========================================
# 1. DEFINE BENCHMARKS
# ==========================================
BENCHMARKS = {
    'ICU': {'nurse': 2, 'doctor': 14, 'alos_min': 2, 'alos_max': 4},
    'surgery': {'nurse': 6, 'doctor': 15, 'alos_min': 1, 'alos_max': 8},
    'general_medicine': {'nurse': 6, 'doctor': 15, 'alos_min': 4, 'alos_max': 6},
    'emergency': {'nurse': 4, 'doctor': None, 'alos_min': 4/24, 'alos_max': 6/24}
}

# ==========================================
# 2. LOAD AND PROCESS DATA
# ==========================================
try:
    services_df = pd.read_csv('services_weekly.csv')
    staff_schedule_df = pd.read_csv('staff_schedule.csv')
    patients_df = pd.read_csv('patients.csv')
except FileNotFoundError:
    print("Error: CSV files not found. Please ensure 'services_weekly.csv', 'staff_schedule.csv', and 'patients.csv' are in the directory.")
    exit()

# Sanitize strings
services_df['service'] = services_df['service'].str.strip()
staff_schedule_df['service'] = staff_schedule_df['service'].str.strip()
patients_df['service'] = patients_df['service'].str.strip()

# --- PREPARE STAFF COUNTS (WEEKLY) ---
present_staff = staff_schedule_df[staff_schedule_df['present'] == 1]
staff_counts = present_staff.groupby(['week', 'service', 'role']).size().unstack(fill_value=0).reset_index()
for role in ['doctor', 'nurse']:
    if role not in staff_counts.columns: staff_counts[role] = 0

# --- PREPARE WEEKLY DATA (Standard Line Charts) ---
merged_df = pd.merge(services_df[['week', 'month', 'service', 'patients_admitted']], staff_counts, on=['week', 'service'], how='left')
merged_df['doctor'] = merged_df['doctor'].fillna(0)
merged_df['nurse'] = merged_df['nurse'].fillna(0)

# Fix Month Calculation for Weekly Data
merged_df['month'] = (pd.Timestamp('2025-01-01') + pd.to_timedelta((merged_df['week'] - 1) * 7, unit='D')).dt.month
merged_df['quarter'] = ((merged_df['month'] - 1) // 3) + 1
merged_df['week_of_month'] = merged_df.groupby('month')['week'].rank(method='dense').astype(int)

# ALOS Calculation
patients_df['arrival_date'] = pd.to_datetime(patients_df['arrival_date'])
patients_df['departure_date'] = pd.to_datetime(patients_df['departure_date'])
patients_df['los'] = (patients_df['departure_date'] - patients_df['arrival_date']).dt.days
patients_df['week'] = patients_df['arrival_date'].dt.isocalendar().week

alos_weekly = patients_df.groupby(['week', 'service'])['los'].mean().reset_index(name='avg_los')
merged_df = pd.merge(merged_df, alos_weekly, on=['week', 'service'], how='left')
merged_df['avg_los'] = merged_df['avg_los'].fillna(0)

available_services = sorted(merged_df['service'].unique())

# Ratio Calculation Helper
def calculate_ratios(df, role):
    col_name = f'{role}_ratio'
    def get_ratio(row):
        patients = row['patients_admitted']
        staff = row[role]
        if staff > 0: 
            return patients / staff
        else:
            return 0 
    df[col_name] = df.apply(get_ratio, axis=1)
    df[f'{role}_crisis'] = (df[role] == 0) & (df['patients_admitted'] > 0)
    return df

merged_df = calculate_ratios(merged_df, 'nurse')
merged_df = calculate_ratios(merged_df, 'doctor')

# --- PREPARE DAILY DATA ---
patients_df['day_name'] = patients_df['arrival_date'].dt.day_name()
daily_patients = patients_df.groupby(['week', 'day_name', 'service']).size().reset_index(name='patients_admitted')
daily_merged = pd.merge(daily_patients, staff_counts, on=['week', 'service'], how='left')
daily_merged['doctor'] = daily_merged['doctor'].fillna(0)
daily_merged['nurse'] = daily_merged['nurse'].fillna(0)

# Add Month and Quarter columns to Daily Data
daily_merged['month'] = (pd.Timestamp('2025-01-01') + pd.to_timedelta((daily_merged['week'] - 1) * 7, unit='D')).dt.month
daily_merged['quarter'] = ((daily_merged['month'] - 1) // 3) + 1

daily_merged = calculate_ratios(daily_merged, 'nurse')
daily_merged = calculate_ratios(daily_merged, 'doctor')
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_merged['day_name'] = pd.Categorical(daily_merged['day_name'], categories=days_order, ordered=True)
daily_merged = daily_merged.sort_values(['week', 'day_name'])

# ==========================================
# 3. APP LAYOUT
# ==========================================
app = dash.Dash(__name__)
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            .flip-container { perspective: 1000px; position: relative; width: 100%; height: 380px; }
            .flipper { transition: 0.8s; transform-style: preserve-3d; position: relative; width: 100%; height: 100%; }
            .flip-container.flipped .flipper { transform: rotateY(180deg); }
            .front, .back { backface-visibility: hidden; position: absolute; top: 0; left: 0; width: 100%; height: 100%; background-color: white; }
            .front { z-index: 2; transform: rotateY(0deg); }
            .back { transform: rotateY(180deg); }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>{%config%}{%scripts%}{%renderer%}</footer>
    </body>
</html>
'''

app.layout = html.Div([
    html.H1("Hospital Clinical Dashboard", style={'textAlign': 'center', 'fontFamily': 'Arial'}),
    
    # CONTROLS
    html.Div([
        html.Div([
            html.Label("1. Service:", style={'fontWeight': 'bold'}),
            dcc.Dropdown(id='service-dropdown', options=[], value='surgery', clearable=False)
        ], style={'width': '22%'}),
        
        html.Div([
            html.Label("2. Time Scale:", style={'fontWeight': 'bold'}),
            dcc.Dropdown(
                id='time-scale-dropdown',
                options=[{'label': 'Weekly', 'value': 'week'}, {'label': 'Monthly', 'value': 'month'}, {'label': 'Quarterly', 'value': 'quarter'}],
                value='week', clearable=False
            )
        ], style={'width': '22%'}),
        
        # View Type Container (Targeted by callback to toggle visibility)
        html.Div([
            html.Label("3. View Type:", style={'fontWeight': 'bold', 'color': '#d62728'}),
            dcc.Dropdown(
                id='view-type-dropdown',
                options=[{'label': 'Line Chart', 'value': 'line'}, {'label': 'Heatmap', 'value': 'heatmap'}],
                value='line', clearable=False
            )
        ], id='view-type-container', style={'width': '22%', 'display': 'none'}), # Default hidden for Weekly
        
        html.Div([
            html.Label("4. Role:", style={'fontWeight': 'bold'}),
            dcc.RadioItems(
                id='role-selector',
                options=[{'label': ' Nurse', 'value': 'nurse'}, {'label': ' Doctor', 'value': 'doctor'}],
                value='nurse', inline=True
            )
        ], style={'width': '28%', 'paddingTop': '10px'})
    ], style={'width': '95%', 'margin': '0 auto 10px auto', 'display': 'flex', 'justifyContent': 'space-between'}),

    # TIME INTERVAL SLIDER
    html.Div([
        html.Label("Select Time Interval:", style={'fontWeight': 'bold'}),
        dcc.RangeSlider(
            id='time-range-slider',
            step=1,
            marks=None,
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={'width': '95%', 'margin': '0 auto 30px auto', 'padding': '10px', 'backgroundColor': '#f9f9f9', 'borderRadius': '5px'}),

    # ROW 1
    html.Div([
        html.Div([
            html.Div(id='flip-container', className='flip-container', children=[
                html.Div(className='flipper', children=[
                    html.Div(className='front', children=[
                        dcc.Graph(id='trend-chart', hoverData=None, clear_on_unhover=True, style={'height': '100%'})
                    ]),
                    html.Div(className='back', children=[
                        dcc.Graph(id='heatmap-chart', hoverData=None, clear_on_unhover=True, style={'height': '100%'})
                    ])
                ])
            ])
        ], style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top', 'border': '1px solid #ccc', 'padding': '5px', 'height': '380px'}),
        
        html.Div([
            dcc.Graph(id='deviation-chart', style={'height': '370px'})
        ], style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top', 'float': 'right', 'border': '1px solid #ccc', 'padding': '5px', 'height': '380px'})
    ], style={'width': '95%', 'margin': '0 auto'}),

    # ROW 2
    html.Div([
        dcc.Graph(id='alos-chart')
    ], style={'width': '95%', 'margin': '20px auto', 'border': '1px solid #ccc', 'padding': '10px'}),
])

# ==========================================
# 5. CALLBACKS
# ==========================================

# 1. Update Service Options
@app.callback(
    [Output('service-dropdown', 'options'),
     Output('service-dropdown', 'value')],
    Input('role-selector', 'value'),
    State('service-dropdown', 'value')
)
def update_service_options(role, current_service):
    valid_services = [s for s in available_services if BENCHMARKS.get(s, {}).get(role) is not None]
    options = [{'label': s.replace('_', ' ').title(), 'value': s} for s in valid_services]
    
    new_value = current_service
    if current_service not in valid_services:
        new_value = valid_services[0] if valid_services else None
        
    return options, new_value

# 2. Update View Type Options, Value AND Visibility based on Time Scale
@app.callback(
    [Output('view-type-dropdown', 'options'),
     Output('view-type-dropdown', 'value'),
     Output('view-type-container', 'style')],
    [Input('time-scale-dropdown', 'value')],
    [State('view-type-dropdown', 'value')]
)
def update_view_options_and_visibility(time_scale, current_view):
    options_all = [{'label': 'Line Chart', 'value': 'line'}, {'label': 'Heatmap', 'value': 'heatmap'}]
    options_line = [{'label': 'Line Chart', 'value': 'line'}]
    
    # Logic: Only show heatmap option and container if NOT Weekly
    if time_scale == 'week':
        return options_line, 'line', {'display': 'none'}
    else:
        return options_all, current_view, {'width': '22%', 'display': 'block'}

# 3. Flip Animation Class
@app.callback(
    Output('flip-container', 'className'),
    [Input('view-type-dropdown', 'value')]
)
def update_flip_animation(view_type):
    return 'flip-container flipped' if view_type == 'heatmap' else 'flip-container'

# 4. Update Slider Range
@app.callback(
    [Output('time-range-slider', 'min'),
     Output('time-range-slider', 'max'),
     Output('time-range-slider', 'value'),
     Output('time-range-slider', 'marks')],
    Input('time-scale-dropdown', 'value')
)
def update_slider_config(time_scale):
    if time_scale == 'week':
        min_v, max_v = 1, 52
    elif time_scale == 'month':
        min_v, max_v = 1, 12
    elif time_scale == 'quarter':
        min_v, max_v = 1, 4
    
    marks = {i: str(i) for i in range(min_v, max_v+1, 5)} if max_v > 20 else {i: str(i) for i in range(min_v, max_v+1)}
    return min_v, max_v, [min_v, max_v], marks

# 5. Update Charts
@app.callback(
    [Output('trend-chart', 'figure'),
     Output('heatmap-chart', 'figure'),
     Output('alos-chart', 'figure'),
     Output('deviation-chart', 'figure')],
    [Input('service-dropdown', 'value'),
     Input('role-selector', 'value'),
     Input('time-scale-dropdown', 'value'),
     Input('time-range-slider', 'value'),
     Input('trend-chart', 'hoverData'),
     Input('heatmap-chart', 'hoverData'),
     Input('view-type-dropdown', 'value')]
)
def update_charts(service, role, time_scale, slider_range, hover_line, hover_heat, view_type):
    if not service:
        return [go.Figure().update_layout(title="Select Service")] * 4
        
    limits = BENCHMARKS.get(service, {})
    ratio_limit = limits.get(role)
    if ratio_limit is None:
        return dash.no_update, dash.no_update, dash.no_update, dash.no_update

    # Base DF (Weekly aggregated)
    df = merged_df[merged_df['service'] == service].copy()
    
    # Daily DF (For Heatmap)
    df_daily = daily_merged[daily_merged['service'] == service].copy()

    # --- FILTER DATA ---
    if slider_range:
        start, end = slider_range
        if start == end:
            max_limit = 52 if time_scale == 'week' else (12 if time_scale == 'month' else 4)
            if end < max_limit: end += 1
            else: start = max(1, start - 1)
        
        if time_scale == 'week':
            df = df[(df['week'] >= start) & (df['week'] <= end)]
            df_daily = df_daily[(df_daily['week'] >= start) & (df_daily['week'] <= end)]
        elif time_scale == 'month':
            df = df[(df['month'] >= start) & (df['month'] <= end)]
            df_daily = df_daily[(df_daily['month'] >= start) & (df_daily['month'] <= end)]
        elif time_scale == 'quarter':
            df = df[(df['quarter'] >= start) & (df['quarter'] <= end)]
            df_daily = df_daily[(df_daily['quarter'] >= start) & (df_daily['quarter'] <= end)]
    
    # Process Aggregation for Line Chart
    if time_scale == 'week':
        plot_df = df
        x_col = 'week'
        time_label = "Week"
    else:
        plot_df = df.groupby(time_scale).agg({
            'patients_admitted': 'mean',
            'nurse': 'mean',
            'doctor': 'mean',
            'avg_los': 'mean'
        }).reset_index()
        plot_df = calculate_ratios(plot_df, 'nurse')
        plot_df = calculate_ratios(plot_df, 'doctor')
        x_col = time_scale
        time_label = time_scale.title()
        
        if time_scale == 'month':
            plot_df['label'] = pd.to_datetime(plot_df['month'], format='%m').dt.month_name().str.slice(stop=3)
        elif time_scale == 'quarter':
            plot_df['label'] = 'Q' + plot_df['quarter'].astype(str)

    x_data = plot_df['label'] if 'label' in plot_df.columns else plot_df[x_col]
    
    alos_min, alos_max = limits.get('alos_min', 0), limits.get('alos_max', 10)
    y_col_ratio = f'{role}_ratio'
    title_role = role.title()

    # Hover Logic
    hover_index = None
    if time_scale != 'week' and view_type == 'heatmap' and hover_heat:
        try:
            hover_x_label = hover_heat['points'][0]['x']
            if 'label' in plot_df.columns:
                indices = plot_df.index[plot_df['label'] == hover_x_label].tolist()
            else:
                indices = plot_df.index[plot_df[x_col] == hover_x_label].tolist()
            if indices: hover_index = indices[0]
        except: pass
    elif hover_line:
        try: hover_index = hover_line['points'][0]['pointIndex']
        except: pass

    # --- 1. LINE CHART ---
    fig_trend = go.Figure()
    valid_max = plot_df[y_col_ratio].max() if not plot_df.empty else 0
    if pd.isna(valid_max) or valid_max == 0: valid_max = ratio_limit
    y_range_max = max(valid_max, ratio_limit * 1.5)
    x_list = x_data.tolist() if hasattr(x_data, 'tolist') else list(x_data)
    
    # Apply updated hovertemplate for Staffing Ratio Line Chart
    hover_template_trend = f"{time_label}: %{{x}}<br>Avg. Load: %{{y:.2f}}<extra></extra>"

    fig_trend.add_trace(go.Scatter(x=x_list, y=[ratio_limit]*len(x_list), mode='lines', line=dict(width=0), fill='tozeroy', fillcolor='rgba(232, 245, 233, 1.0)', name='Safe Zone', hoverinfo='skip', showlegend=False))
    fig_trend.add_trace(go.Scatter(x=x_list, y=[y_range_max*1.3]*len(x_list), mode='lines', line=dict(width=0), fill='tonexty', fillcolor='rgba(255, 235, 238, 1.0)', name='Critical Zone', hoverinfo='skip', showlegend=False))
    fig_trend.add_trace(go.Scatter(x=x_data, y=plot_df[y_col_ratio], mode='lines+markers', name=f'{title_role} Load', line=dict(color='#2c3e50', width=3), marker=dict(color='#2c3e50', size=8), hovertemplate=hover_template_trend))
    
    if hover_index is not None and hover_index < len(x_data) and (time_scale == 'week' or view_type != 'heatmap'):
        fig_trend.add_vline(x=x_data.iloc[hover_index] if hasattr(x_data, 'iloc') else list(x_data)[hover_index], line_width=1, line_dash="dash", line_color="black")

    fig_trend.update_layout(title=f"<b>Average patients to {title_role} Ratio</b>", xaxis_title=time_scale.title(), yaxis_title=f"Patients per {title_role}", margin=dict(l=40, r=20, t=40, b=30), height=350, transition={'duration': 800, 'easing': 'cubic-in-out'})

    # --- 2. HEATMAP ---
    fig_heat = go.Figure()
    
    if time_scale != 'week':
        z_matrix = pd.DataFrame()
        x_title, y_title = "", ""
        hover_template_heat = ""
        
        if time_scale == 'month':
            # Y=Week of Month, X=Month
            heat_df = df.groupby(['month', 'week_of_month']).agg({'patients_admitted':'sum', 'nurse':'mean', 'doctor':'mean'}).reset_index()
            heat_df = calculate_ratios(heat_df, role)
            z_matrix = heat_df.pivot(index='week_of_month', columns='month', values=y_col_ratio)
            x_title, y_title = "Month", "Week of Month"
            z_matrix.columns = [pd.to_datetime(m, format='%m').month_name()[:3] for m in z_matrix.columns]
            
            # Hover: Month name, Week number, Load
            hover_template_heat = "Month: %{x}<br>Week: %{y}<br>Avg. Load: %{z:.2f}<extra></extra>"
            
        elif time_scale == 'quarter':
            # Y=Month, X=Quarter
            heat_df = df.groupby(['quarter', 'month']).agg({'patients_admitted':'mean', 'nurse':'mean', 'doctor':'mean'}).reset_index()
            heat_df = calculate_ratios(heat_df, role)
            heat_df['month_name'] = pd.to_datetime(heat_df['month'], format='%m').dt.month_name().str.slice(stop=3)
            z_matrix = heat_df.pivot(index='month_name', columns='quarter', values=y_col_ratio)
            month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
            existing_months = [m for m in month_order if m in z_matrix.index]
            z_matrix = z_matrix.reindex(existing_months)
            z_matrix.columns = ['Q' + str(c) for c in z_matrix.columns]
            x_title, y_title = "Quarter", "Month"
            
            # Hover: Quarter name, Month name, Load
            hover_template_heat = "Quarter: %{x}<br>Month: %{y}<br>Avg. Load: %{z:.2f}<extra></extra>"

        if not z_matrix.empty:
            fig_heat.add_trace(go.Heatmap(
                z=z_matrix.values, 
                x=z_matrix.columns, 
                y=z_matrix.index, 
                colorscale='RdYlGn_r', 
                zmid=ratio_limit,
                xgap=3, ygap=3,
                hovertemplate=hover_template_heat
            ))
            fig_heat.update_layout(title=f"<b>Average patients to {title_role} Ratio</b>", xaxis_title=x_title, yaxis_title=y_title, margin=dict(l=40, r=20, t=40, b=30), height=350)

    # --- 3. ALOS ---
    base_colors = ['#2ca02c' if (alos_min <= x <= alos_max) else '#d62728' for x in plot_df['avg_los']]
    if hover_index is not None and hover_index < len(plot_df):
        final_colors, opacities = ['lightgrey']*len(plot_df), [0.3]*len(plot_df)
        final_colors[hover_index], opacities[hover_index] = base_colors[hover_index], 1.0
    else:
        final_colors, opacities = base_colors, [1.0]*len(plot_df)

    fig_alos = go.Figure()
    
    # Apply updated hovertemplate for ALOS Chart
    hover_template_alos = f"{time_label}: %{{x}}<br>ALOS: %{{y:.2f}}<extra></extra>"
    
    target_text = "Target: 4-6 Hours" if service == 'emergency' else f"Target: {alos_min}-{alos_max} Days"
    fig_alos.add_hrect(y0=alos_min, y1=alos_max, fillcolor="green", opacity=0.1, layer="below", line_width=0, annotation_text=target_text)
    fig_alos.add_trace(go.Scatter(x=x_data, y=plot_df['avg_los'], mode='lines', line=dict(color='lightgrey'), hoverinfo='skip'))
    fig_alos.add_trace(go.Scatter(x=x_data, y=plot_df['avg_los'], mode='markers', marker=dict(color=final_colors, size=12, opacity=opacities), hovertemplate=hover_template_alos))
    
    fig_alos.update_layout(title="<b>Patients Average Length of Stay</b>", xaxis_title=time_scale.title(), yaxis_title="Days", height=350, margin=dict(l=40, r=20, t=40, b=30), showlegend=False, transition={'duration': 800, 'easing': 'cubic-in-out'})

    # --- 4. DEVIATION ---
    plot_df['dev'] = (plot_df[y_col_ratio] - ratio_limit)
    dev_colors = ['#d62728' if x > 0 else '#2ca02c' for x in plot_df['dev']]
    bar_opacities = [1.0 if (hover_index is None or i==hover_index) else 0.3 for i in range(len(plot_df))]

    # Apply updated hovertemplate for Deviation Chart
    hover_template_dev = f"{time_label}: %{{x}}<br>Deviation: %{{y:.2f}}<extra></extra>"

    fig_dev = go.Figure(go.Bar(x=x_data, y=plot_df['dev'], marker_color=dev_colors, marker_opacity=bar_opacities, hovertemplate=hover_template_dev))
    fig_dev.update_layout(title="<b>Staffing Deviation from Benchmark</b>", height=350, margin=dict(l=40, r=20, t=40, b=30))

    return fig_trend, fig_heat, fig_alos, fig_dev

if __name__ == '__main__':
    print("Dashboard running on http://127.0.0.1:8050/")
    app.run(debug=True)

Dashboard running on http://127.0.0.1:8050/
